# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [360]:
# importing needed libraries for the entire notebook in one go
%matplotlib inline
%matplotlib widget
import os
from pathlib import Path
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from mintpy.cli import view, tsview
from itertools import combinations
import eof
import geemap
import csv
from isceobj.XmlUtil import FastXML as xml


# only need these two if you plan on searching andrequesting for data from within this notebook
# in my opinion, while it may take longer, it is much easier to use the ASF Vertex Website
# that said, if you are monitoring a reoccuring site and just need more imagery, searching using the notebook would be best
import asf_search as asf
import hyp3_sdk as sdk

In [361]:
def project_dir(proj_name):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    """

    #creates file on your desktop containing the work of this notebook
    work_dir = Path.home() / 'Desktop' / proj_name
    work_dir.mkdir(parents=True, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = work_dir / 'interferometry'
    if_dir.mkdir(parents=True, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = work_dir / 'time_series'
    ts_dir.mkdir(parents=True, exist_ok=True)
    
    
    ######### ISCE2 Directories for interferometry
    xmls_dir = if_dir / 'xmls'
    xmls_dir.mkdir(parents=True, exist_ok=True)
    
    topsxml_dir = xmls_dir / 'topsApp'
    topsxml_dir.mkdir(parents=True, exist_ok=True)
    
    refxml_dir = xmls_dir / 'reference'
    refxml_dir.mkdir(parents=True, exist_ok=True)
    
    secxml_dir = xmls_dir / 'secondary'
    secxml_dir.mkdir(parents=True, exist_ok=True)
    
    xmldirectories = [topsxml_dir, refxml_dir, secxml_dir]
    
    isceref_dir = if_dir / 'reference'
    isceref_dir.mkdir(parents=True, exist_ok=True)
    
    iscesec_dir = if_dir / 'secondary'
    iscesec_dir.mkdir(parents=True, exist_ok=True)
    
    orbits_dir = if_dir / 'orbits'
    orbits_dir.mkdir(parents=True, exist_ok=True)
    
    ifdirectories = [isceref_dir, iscesec_dir, orbits_dir]
    
    ######### MinPy Directories for time-series
    baseline_dir = ts_dir / 'baseline'
    baseline_dir.mkdir(parents=True, exist_ok=True)
    
    ref_dir = ts_dir / 'reference'
    ref_dir.mkdir(parents=True, exist_ok=True)
    
    merged_dir = ts_dir / 'merged'
    merged_dir.mkdir(parents=True, exist_ok=True)
    
    geomref_dir = merged_dir / 'geom_reference'
    geomref_dir.mkdir(parents=True, exist_ok=True)
    
    interfer_dir = merged_dir / 'interferograms'
    interfer_dir.mkdir(parents=True, exist_ok=True)
    
    sec_dir = ts_dir / 'secondaries'
    sec_dir.mkdir(parents=True, exist_ok=True)

    mintpy_dir = ts_dir / 'mintpy'
    mintpy_dir.mkdir(parents=True, exist_ok=True)
    
    tsdirectories = [baseline_dir, ref_dir, merged_dir, geomref_dir, interfer_dir, sec_dir, mintpy_dir]

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [422]:
def get_triplets(slc_zips):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    slc_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
    slc_list =[im for im in slc_list]
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
    return triplet_dict

In [362]:
# put template file generator here, look for examples for the templates
def config_file(outName, CONFIG_TXT): 
    """
    Write configuration files for MintPy to process ISCE2 product
    outName can be something like proj_name    
    """
    if os.path.isfile(outName):
        with open(outName, "w") as fid:
            fid.write(CONFIG_TXT)

    else:
        with open(outName, "a") as fid:
            fid.write("\n" + CONFIG_TXT)


In [364]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
def ref_sec_xmls(subswaths, slc_zips_dirs, ifdirectories, xmldirectories, isce_aoi):
  """
  subswaths = list(int), a list containing the int subswaths (1, 2, 3)
  slc_zip_dirs = list containing the paths to each SLC.zip file from ASF
  ifdirectories = list containing the paths to the directories used for ISCE2
  xmldirectories = list containing paths to the directories where the xmls will be stored
  isce_aoi = the area of interest drawn in the bbox section
  """


  # subswaths = [1,2]
  for slc in slc_zips_dirs:
      slc_date = slc[-38:-30]
      filetype = ['reference', 'secondary']
      for file in ['reference', 'secondary']:
        if file == 'reference':
          CONFIG_TXT=f"""<?xml version="1.0" encoding="UTF-8"?>
<component name="reference">
  <property name="safe">{slc}</property>
  <property name="swath number">{subswaths}</property>
  <!-- Be careful. Metadata for unpacked product will be stored in "output directory".xml file. So don't use the same name for input catalog file and output directory. You will end up overwriting the input catalog. We use referencedir to ensure we dont overwrite reference.xml -->
  <property name="output directory">{ifdirectories[0]}</property>
  <property name="orbit directory">{ifdirectories[2]}</property>
  <property name="region of interest">{isce_aoi}</property>
</component>
"""
          configName = os.path.join(xmldirectories[1], "{}.xml".format(f'{slc_date}ref'))
          config_file(configName, CONFIG_TXT)
        
        else:
          CONFIG_TXT=f"""<?xml version="1.0" encoding="UTF-8"?>
<component name="secondary">
  <property name="safe">{slc}</property>
  <property name="swath number">{subswaths}</property>
  <!-- Be careful. Metadata for unpacked product will be stored in "output directory".xml file. So don't use the same name for input catalog file and output directory. You will end up overwriting the input catalog. We use referencedir to ensure we dont overwrite reference.xml -->
  <property name="output directory">{ifdirectories[0]}</property>
  <property name="orbit directory">{ifdirectories[2]}</property>
  <property name="region of interest">{isce_aoi}</property>
</component>
"""
          configName = os.path.join(xmldirectories[2], "{}.xml".format(f'{slc_date}sec'))
          config_file(configName, CONFIG_TXT)

In [348]:
# function to create the triplets dictionary and the topsApp.xml files for topsInSAR

def topsAppxml(xmldirectories):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    xmldirectories = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """

    for key in triplet_dict:
        triplet = triplet_dict[key]
        for i, intf in enumerate(triplet):
            insarxmlname = f'{triplet[i][0][-15:-7]}_{triplet[i][1][-15:-7]}topsApp'
            CONFIG_TXT=f"""<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
      <catalog>['{triplet[i][0]}']</catalog>
    </component>
    <component name="secondary">
      <catalog>['{triplet[i][1]}']</catalog>
    </component>
    <property name="range looks">6</property>
    <property name="azimuth looks">2</property>
    <property name="filter strength">0.4</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
  </component>
</topsApp>
"""

            configName = os.path.join(xmldirectories[0], "{}.xml".format(insarxmlname))
            config_file(configName, CONFIG_TXT)
    
    topsapp_list = [os.path.join(xmldirectories[0], name) for name in os.listdir(xmldirectories[0])]
    # topsapp_list = os.listdir(xmldirectories[0])
    return triplet_dict, topsapp_list

# Establish working directroy and data paths

In [365]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'Harris_etal_2023'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name)

In [409]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2019_2020/'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

triplet_dict = get_triplets(slc_zips)

In [429]:
from xml.etree.ElementTree import Element, SubElement, tostring, ElementTree

# Get bbox for your area of interest in the SLC images, faster processing with less data

In [349]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [350]:
# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(m.draw_features[0].getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
aoi = "POLYGON((" + ", ".join([f'{northing} {easting}' for northing, easting in vertices[:]]) +"))"

#create isce compatibile bbox
isce_aoi = [min([easting for northing, easting in vertices[:]]), max([easting for northing, easting in vertices[:]]), min([northing for northing, easting in vertices[:]]), max([northing for northing, easting in vertices[:]])]
print(isce_aoi)

[29.823966, 30.086919, -93.543091, -93.306885]


# Get s-1 orbit files (.EOF) using sentineleof library

In [353]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create reference and secondary .xml files

In [354]:
ref_sec_xmls([1,2], slc_zips_dirs, ifdirectories, xmldirectories, isce_aoi)

# Create topsApp.xml file

In [ ]:
topsApp = Element('topsApp')
component = SubElement(topsApp, 'component', name='topsinsar')

sensor_name = SubElement(component, 'property', name='Sensor name')
sensor_name.text = 'SENTINEL1'

####### Reference.xml
ref = SubElement(component, 'component', name='reference')

safe1 = SubElement(ref, 'property', name='safe')
safe1.text = os.path.join(slc_zips, triplet_dict['triplet_1'][0][0])

swath_num = SubElement(ref, 'property', name='swath number')
swath_num.text = [1,2]

ref_out = SubElement(ref, 'property', name='output directort')
ref_out = xmldirectories[1]

ref_orbit = SubElement(ref, 'property', name='orbit directory')
ref_orbit.text = ifdirectories[2]

roi = SubElement(ref, 'property', name='region of interest')
roi.text = isce_aoi

########## Secondary.xml
sec = SubElement(component, 'component', name='reference')

safe1 = SubElement(sec, 'property', name='safe')
safe1.text = os.path.join(slc_zips, triplet_dict['triplet_1'][0][1])

swath_num = SubElement(sec, 'property', name='swath number')
swath_num.text = [1, 2]

sec_out = SubElement(sec, 'property', name='output directort')
sec_out = xmldirectories[2]

sec_orbit = SubElement(sec, 'property', name='orbit directory')
sec_orbit.text = ifdirectories[2]

roi = SubElement(sec, 'property', name='region of interest')
roi.text = isce_aoi

########## topsApp.xml
range_looks = SubElement(component, 'property', name='range looks')
range_looks.text = 6

azi_looks = SubElement(component, 'property', name='azimuth looks')
azi_looks.text = 2

do_unwrap = SubElement(component, 'property', name='do unwrap')
do_unwrap.text = 'True'

unwrap_name = SubElement(component, 'property', name='unwrapper name')
unwrap_name.text ='snaphu_mcf'

# Create XML tree and save to file
tree = ElementTree(topsApp)
# xml_filename = os.path.join('/home/wcc', 'topsApp.xml')
tree.write('topsApp.xml', encoding="utf-8", xml_declaration=True)

TypeError: write() argument must be str, not list

In [ ]:

# Function to create a topsApp.xml for each SLC pair
def create_topsapp_xml(slc1_dir, slc2_dir, output_dir):
    topsApp = Element('topsApp')
    component = SubElement(topsApp, 'component', name='topsinsar')
    
    sensor_name = SubElement(component, 'property', name='sensor name')
    sensor_name.text = 'SENTINEL1'
    
    safe1 = SubElement(component, 'property', name='safe')
    safe1.text = slc1_dir
    
    safe2 = SubElement(component, 'property', name='safe')
    safe2.text = slc2_dir
    
    output_directory = SubElement(component, 'property', name='output directory')
    output_directory.text = output_dir
    
    do_unwrap = SubElement(component, 'property', name='do unwrap')
    do_unwrap.text = 'True'
    
    # Create XML tree and save to file
    tree = ElementTree(topsApp)
    xml_filename = os.path.join(output_dir, 'topsApp.xml')
    tree.write(xml_filename)

# Example usage for each pair
for idx, (slc1_dir, slc2_dir) in enumerate([("SLC1_folder", "SLC2_folder"), ("SLC2_folder", "SLC3_folder")]):
    output_dir = f"interferogram_{idx+1}"
    os.makedirs(output_dir, exist_ok=True)
    create_topsapp_xml(slc1_dir, slc2_dir, output_dir)


In [355]:
# create new triplets dictionary with new triplets list
# triplets list will be [ifij, ifjk, ifki, ifik]
# which could be [(refi.xml, seck.xml), (refj.xml, seck.xml), (refk.xml, seci.xml), (refi.xml, seck.xml)]



In [301]:
#for future reference
if_ij = triplet_dict['triplet_1'][0]
if_jk = triplet_dict['triplet_1'][1]
if_ki = triplet_dict['triplet_1'][2]
if_ik = triplet_dict['triplet_1'][3]

# topsApp interferometry (finally!)

In [ ]:
# %%   batch run topsApp.py
ifg_pairs = load_interferogram_pairs(pair_file)
mapper = mapper_date_data(sar_dir, suffix='zip')

for i, ifg in enumerate(tqdm(ifg_pairs)):
    reference, secondary = ifg
    reference_file = os.path.join(sar_dir, mapper[reference])
    secondary_file = os.path.join(sar_dir, mapper[secondary])

    ifg_name = f'{reference}_{secondary}'
    ifg_dir = home_dir / f'interferogams/{ifg_name}'
    if not ifg_dir.is_dir():
        ifg_dir.mkdir(parents=True)

    # create xml file
    reference_dir = ifg_dir / f'{reference}'
    secondary_dir = ifg_dir / f'{secondary}'
    xml_file = ifg_dir / 'topsApp.xml'

    generate_topsApp_xml(reference_dir, secondary_dir,
                         reference_file, secondary_file, roi, xml_file)

    # cd ifg folder
    os.chdir(ifg_dir)

    # link dem file to current folder
    print(ifg_name)
    cmd_link = f'ln -sf {str(Path(dem).parent)}/* ./'
    os.system(cmd_link)

    # execute topsApp.py
    cmd_tops = f'topsApp.py {xml_file} --start=startup'
    logger.info(cmd_tops)
    os.system(cmd_tops)

In [356]:
print(topsapp_list[0])

/home/wcc/Desktop/Harris_etal_2023/interferometry/xmls/topsApp/20200410_20200422topsApp.xml


In [303]:
import isce
from topsApp import TopsInSAR
a= TopsInSAR.step(a, name ='startup')

AttributeError: 'NoneType' object has no attribute 'step_list_help'

In [358]:
%run ~/tools/miniforge//envs/insar/lib/python3.11/site-packages/isce/applications/topsApp.py /home/wcc/Desktop/Harris_etal_2023/interferometry/xmls/topsApp/20200410_20200422topsApp.xml

2024-09-13 14:20:01,248 - root - ERROR - Error. catalog must be a filename or  a dictionary


RuntimeError: No active exception to reraise

In [340]:
%run ~/tools/miniforge//envs/insar/lib/python3.11/site-packages/isce/applications/topsApp.py --dostep='preprocess'

2024-09-13 14:13:45,893 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
Step processing
Running step preprocess


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [339]:
%run ~/tools/miniforge//envs/insar/lib/python3.11/site-packages/isce/applications/iscehelp.py -i


Type AmpImage: constructor requires no arguments

Type Attitude: constructor requires no arguments

Type Correct: constructor requires no arguments

Type Correct_geoid_i2_srtm: constructor requires no arguments

Type DataManager: Constructor requires arguments described in the
table below. Use the -a option with the mandatory arguments
to ask for more help. Run iscehelp.py -h for more info on the -a option.

name              type       argtype    mandatory  values               default   
================= ========== ========== ========== ==================== ==========
0                 str        positional True       dem1                 None      
                                                   dem2                           
                                                   wbd                            


Type DataRetriever: constructor requires no arguments

Type DataTileManager: constructor requires no arguments

Type DemImage: constructor requires no arguments

Type Dem

In [279]:
print(a)

None


In [ ]:
a = Insar(name='topsApp', cmdline=f'{topsapp_list[0]}') # different XML for each interferogram? Will need a lot to form all the triplets
a.configure()
# a.run()

In [1]:
# use something to replicate wget https://sar-mpc.eu/download/ca97845e-1314-4817-91d8-f39afbeff74d/ -O S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.zip
# iterate through all .zip files
from mintpy import prep_isce

In [29]:
import isce.applications


print(isce.applications)

<module 'isce.applications' from '/home/wcc/tools/miniforge/envs/insar/lib/python3.11/site-packages/isce/applications/__init__.py'>


In [37]:
!python ~/tools/isce2/src/isce2/contrib/topsStack/stackSentinel.py -h

python: can't open file '/home/wcc/tools/isce2/src/isce2/contrib/topsStack/stackSentinel.py': [Errno 2] No such file or directory


In [ ]:
stackSentinel.py

In [35]:
from isce import contrib

ImportError: cannot import name 'contrib' from 'isce' (/home/wcc/tools/miniforge/envs/insar/lib/python3.11/site-packages/isce/__init__.py)